In [1]:
# https://github.com/Kaggle/kaggle-environments/blob/master/kaggle_environments/envs/hungry_geese/hungry_geese.py

In [1]:
from kaggle_environments.envs.hungry_geese.hungry_geese import Observation, Configuration, Action, \
                                                                row_col, adjacent_positions, translate, min_distance

from kaggle_environments import make
from random import choice
import numpy as np
from copy import deepcopy
import pickle

Loading environment football failed: No module named 'gfootball'


In [2]:
import numpy as np
import random
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam

from collections import deque

In [3]:
row_col(40, 11)

(3, 7)

In [4]:
class GreedyAgent:
    def __init__(self):
        
        self.last_action = None
        self.observations = []

    def __call__(self, observation: Observation, configuration: Configuration):
        self.configuration = configuration
        
        board = np.zeros(self.configuration.rows*self.configuration.columns)
        board_shape = (self.configuration.rows, self.configuration.columns)
        
        board_heads = deepcopy(board)
        board_bodies = deepcopy(board)
        board_rewards = deepcopy(board)
        
        
        rows, columns = self.configuration.rows, self.configuration.columns

        food = observation.food
        geese = observation.geese
        
        
        opponents = [
            goose
            for index, goose in enumerate(geese)
            if index != observation.index and len(goose) > 0
        ]

        
        opponent_heads = [opponent[0] for opponent in opponents]
        # Don't move adjacent to any heads
        head_adjacent_positions = {
            opponent_head_adjacent
            for opponent_head in opponent_heads
            for opponent_head_adjacent in adjacent_positions(opponent_head, columns, rows)
        }
        
        tail_adjacent_positions ={
            opponent_tail_adjacent
            for opponent in opponents
            for opponent_tail in [opponent[-1]]
            for opponent_tail_adjacent in adjacent_positions(opponent_tail, columns, rows)
        }
        # Don't move into any bodies
        #bodies, heads = [position for goose in geese for position in goose]
        
        heads = [i[0] for i in geese if len(i)>1]
        bodies = [item for sublist in geese for item in sublist]
        
        board_bodies[list(bodies)] = 1
        board_heads[heads] = 1

        # Move to the closest food
        position = geese[observation.index][0]
        actions = {
            action: min_distance(new_position, food, columns)
            for action in Action
            for new_position in [translate(position, action, columns, rows)]
            if (
                new_position not in head_adjacent_positions and
                new_position not in bodies and
                (self.last_action is None or action != self.last_action.opposite())
            )
        }

        action = min(actions, key=actions.get) if any(actions) else choice([action for action in Action])
        
        
        cur_obs = {}
        cur_obs['head_adjacent_positions'] = head_adjacent_positions
        cur_obs['bodies'] = bodies
        cur_obs['board_bodies'] = board_bodies.reshape(board_shape)
        cur_obs['board_heads'] = board_heads.reshape(board_shape)
        cur_obs['tails'] = tail_adjacent_positions
        cur_obs['actions'] = actions
        cur_obs['action'] = action
        cur_obs['last_action'] = self.last_action
#         cur_obs['goose_size'] = player_goose_len
#         cur_obs['board'] = board
        cur_obs['cur_action'] = action
        self.observations.append(cur_obs)
        
        self.last_action = action
        return action.name


cached_greedy_agents = {}


def greedy_agent(obs, config):
    index = obs["index"]
    if index not in cached_greedy_agents:
        cached_greedy_agents[index] = GreedyAgent(Configuration(config))
    return cached_greedy_agents[index](Observation(obs))

In [5]:
def geese_heads(obs_dict, config_dict):
    """
    Return the position of the geese's heads
    """
    configuration = Configuration(config_dict)

    observation = Observation(obs_dict)
    player_index = observation.index
    player_goose = observation.geese[player_index]
    player_head = player_goose[0]
    player_row, player_column = row_col(player_head, configuration.columns)
    positions = []
    for geese in observation.geese:
        if len(geese)>0:
            geese_head = geese[0]
            row, column = row_col(geese_head, configuration.columns)
        else:
            row = None
            column = None
        positions.append((row, column))
    return positions

def get_last_actions(previous_geese_heads, heads_positions):

    def get_last_action(prev, cur):
        last_action = None

        prev_row = prev[0]
        prev_col = prev[1]
        cur_row = cur[0]
        cur_col = cur[1]

        if cur_row is not None:
            if (cur_row-prev_row == 1) | ((cur_row==0) & (prev_row==6)):
                last_action = Action.SOUTH.name
            elif (cur_row-prev_row == -1) | ((cur_row==6) & (prev_row==0)):
                last_action = Action.NORTH.name
            elif (cur_col-prev_col == 1) | ((cur_col==0) & (prev_col==10)):
                last_action = Action.EAST.name
            elif (cur_col-prev_col == -1) | ((cur_col==10) & (prev_col==0)):
                last_action = Action.WEST.name

        return last_action

    if len(previous_geese_heads) == 0:
        actions = [Action.SOUTH.name, Action.NORTH.name, Action.EAST.name, Action.WEST.name]
        nb_geeses = len(heads_positions)
        last_actions = [actions[np.random.randint(4)] for _ in range(nb_geeses)]
    else:   
        last_actions = [get_last_action(*pos) for pos in zip(previous_geese_heads, heads_positions)]

    return last_actions
    
def central_state_space(obs_dict, config_dict, prev_head):
    """
    Recreating a board where my agent's head in the middle of the board 
    (position (4,5)), and creating features accordingly
    """
    
    configuration = Configuration(config_dict)

    observation = Observation(obs_dict)
    player_index = observation.index
    player_goose = observation.geese[player_index]
    if len(player_goose)==0:
        player_head = prev_head
    else:
        player_head = player_goose[0]
    player_row, player_column = row_col(player_head, configuration.columns)
    row_offset = player_row - 3
    column_offset = player_row - 5

    foods = observation['food']

    def centralize(row, col):
        if col > player_column:
            new_col = (5 + col - player_column) % 11
        else:
            new_col = 5 - (player_column - col)
            if new_col < 0:
                new_col += 11

        if row > player_row:
            new_row = (3 + row - player_row) % 7
        else:
            new_row = 3 - (player_row - row)
            if new_row < 0:
                new_row += 7
        return new_row, new_col

    food1_row, food1_column = centralize(*row_col(foods[0], configuration.columns))
    food2_row, food2_column = centralize(*row_col(foods[1], configuration.columns))

#     food1_row_feat = float(food1_row - 3)/3 if food1_row>=3 else float(food1_row - 3)/3
#     food2_row_feat = float(food2_row - 3)/3 if food2_row>=3 else float(food2_row - 3)/3

#     food1_col_feat = float(food1_column - 5)/5 if food1_column>=5 else float(food1_column - 5)/5
#     food2_col_feat = float(food2_column - 5)/5 if food2_column>=5 else float(food2_column - 5)/5

    # Create the grid
    board = np.zeros([7, 11])
    # Add food to board
    board[food1_row, food1_column] = 1
    board[food2_row, food2_column] = 1

    for goose in observation.geese:
        if len(goose)>0:
            for pos in goose:
                # set bodies to 1
                row, col = centralize(*row_col(pos, configuration.columns))
                board[row, col]=-0.5
                # Set head to five
            row, col = centralize(*row_col(goose[0], configuration.columns))
            board[row,col]=-1
            # Just make sure my goose head is 0
            board[3, 5] = 0
                    
            
    return board #,len(player_goose), food1_row_feat, food1_col_feat, food2_row_feat, food2_col_feat

In [6]:
env = make("hungry_geese", debug=True)
config = env.configuration

state_dict = env.reset(num_agents=4)[0]
observation = state_dict['observation']
action = state_dict['action']

prev_head = 0

In [7]:
central_state_space(observation, config, prev_head)

array([[ 0.,  0.,  0.,  0.,  0., -1.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0., -1.,  0.,  0.,  0.,  0.,  0.]])

In [8]:
env.render(mode="ipython")

In [9]:
def min_dir(p1, p2, max_p):
    """
    min distance and direction from p1
    """
    direction = 'left' # left by default
    d1 = abs(p1 - p2) # Distance going across board
    d2 = min(abs(p1-0), abs(p1 - max_p)) + min(abs(p2-0), abs(p2 - max_p)) # Distance wrapping around board
    
    if p1>p2 and d1<d2:
        direction = 'left'
        
    elif p1>p2 and d1>d2:
        direction ='right'
    
    elif p2>p1 and d1<d2:
        direction = 'right'
    
    elif p2>p1 and d1>d2:
        direction = 'left'
    
    dir_vec = np.zeros(2)
        
    if direction=='left':
        dir_vec[0]=1
    
    else:
        dir_vec[1]=1
    
    min_dist = np.array([min(d1, d2)])/max_p
    return dir_vec, min_dist
        

### Test min dir

In [120]:
cols = 11
p1 = 2
p2 = 5

min_dir(p1,p2,cols)

(array([0., 1.]), array([0.27272727]))

In [121]:
cols = 11
p1 = 2
p2 = 9

min_dir(p1,p2,cols)

(array([1., 0.]), array([0.36363636]))

In [122]:
cols = 11
p1 = 9
p2 = 2

min_dir(p1, p2, cols)

(array([0., 1.]), array([0.36363636]))

In [10]:
class StateTranslator_Compressed:
    def __init__(self):
        
        self.last_action = 'NORTH'
        self.step_count = 0
        self.last_goose_length = 1
        self.observations = []
        
    def set_last_action(self, last_action):
        self.last_action = last_action
        
    def update_step(self):
        self.step_count += 1
        
    def __get_last_action_vec(self):
        action_vec = np.zeros(4)
        
        if self.last_action == 'NORTH':
            action_vec[0] = 1
        elif self.last_action == 'SOUTH':
            action_vec[1] = 1
        elif self.last_action == 'EAST':
            action_vec[2] = 1
        elif self.last_action == 'WEST':
            action_vec[3] = 1
        
        return action_vec

    def get_state(self, observation, configuration):
        self.configuration = configuration
        
        board = np.zeros(self.configuration.rows*self.configuration.columns)
        board_shape = (self.configuration.rows, self.configuration.columns)

        rows, columns = self.configuration.rows, self.configuration.columns
        

        food = observation.food
        food1x, food1y = row_col(food[0], columns)
        food2x, food2y = row_col(food[1], columns)
        
        
        geese = observation.geese
        
        # If my goose died and I need the state, I return 0 instead of having this break... i probably should use if else..
        try:
            my_goose = np.array(geese[observation['index']][0])
        except:
            my_goose = 0
            
        goose_x, goose_y = row_col(my_goose, columns)
            
        self.current_goose_length = len(geese[observation['index']])  
        
        
        def where_da_food_at(goose, food):
            
            food1x, food1y = row_col(food[0], 11)
            food2x, food2y = row_col(food[1], 11)
            
            my_goose_x, my_goose_y = row_col(my_goose, 11)
            
            x_dir1, x_dist1 = min_dir(my_goose_x, food1x, 11)
            x_dir2, x_dist2 = min_dir(my_goose_x, food2x, 11)
            y_dir1, y_dist1 = min_dir(my_goose_y, food1y, 7)
            y_dir2, y_dist2 = min_dir(my_goose_y, food2y, 7)
            
            # Find the food that is the closest
            if (x_dist1+y_dist1) > (x_dist2 + y_dist2):
                x_dir_fin, x_dist_fin = x_dir2, x_dist2
                y_dir_fin, y_dist_fin = y_dir2, y_dist2
                
            else:
                x_dir_fin, x_dist_fin = x_dir1, x_dist1
                y_dir_fin, y_dist_fin = y_dir1, y_dist1
            
            dir_vec = np.concatenate((x_dir_fin, x_dist_fin, y_dir_fin, y_dist_fin))
            
            return dir_vec
        
        dir_vec = where_da_food_at(my_goose, food)
        
        opponents = [
            goose
            for index, goose in enumerate(geese)
            if index != observation.index and len(goose) > 0
        ]
        
        opponent_heads = [opponent[0] for opponent in opponents]
        # Don't move adjacent to any heads
        head_adjacent_positions = {
            opponent_head_adjacent
            for opponent_head in opponent_heads
            for opponent_head_adjacent in adjacent_positions(opponent_head, columns, rows)
        }
        
        

        heads = [i[0] for i in geese if len(i)>1]
        bodies = [item for sublist in geese for item in sublist]
        
        
        board[list(bodies)] = 0.5
        board[heads] = 1
        
        state = np.array([])
        state = np.append(state, self.__get_last_action_vec())
        state = np.append(state, self.step_count/200)
        state = np.append(state, my_goose/(7*11)) # Scale this down by board sie
        state = np.append(state, self.current_goose_length)
        state = np.append(state, dir_vec/10)
        state = np.append(state, board)
        state = state.flatten()
        
        test_state = np.array([])
        test_state = np.append(test_state, dir_vec)
        test_state = np.append(test_state, self.__get_last_action_vec())
        
        return state
    
    def translate_action_to_text(self, action):
        
        h = {0 : 'WEST',
             1: 'EAST',
             2: 'NORTH',
             3: 'SOUTH'}
        
        return h[action]

    def translate_text_to_int(self, action):
        h = {'WEST':0,
            'EAST':1,
            'NORTH':2,
            'SOUTH':3}
        
        return h[action]
    
    def calculate_reward(self):
        prev = self.last_goose_length
        cur = self.current_goose_length
        
        reward = 0
#         if prev>cur:
#             reward = -100
        
        if cur>prev:
            reward = 100
        
        else:
            reward = -1
        
        return reward
    
    def update_length(self):
        self.last_goose_length = self.current_goose_length

In [11]:
class StateTranslator_Central:
    """
    Returns a board where we are always at the center
    """
    
    def __init__(self):
        
        self.last_action = None
        self.step_count = 0
        self.last_goose_length = 1
        self.last_goose_ind = 0
        self.observations = []
        
    def set_last_action(self, last_action):
        self.last_action = last_action
        
    def update_step(self):
        self.step_count += 1
        
    def __get_last_action_vec(self):
        action_vec = np.zeros(4)
        
        if self.last_action == 'NORTH':
            action_vec[0] = 1
        elif self.last_action == 'SOUTH':
            action_vec[1] = 1
        elif self.last_action == 'EAST':
            action_vec[2] = 1
        elif self.last_action == 'WEST':
            action_vec[3] = 1
        
        return action_vec
    
    def translate_action_to_text(self, action):
        
        h = {0 : 'WEST',
             1: 'EAST',
             2: 'NORTH',
             3: 'SOUTH'}
        
        return h[action]

    def translate_text_to_int(self, action):
        h = {'WEST':0,
            'EAST':1,
            'NORTH':2,
            'SOUTH':3}
        
        return h[action]
    
    
    def update_length(self):
        self.last_goose_length = self.current_goose_length
        
    
    def get_state(self, observation, config):
        
        board = central_state_space(observation, config, self.last_goose_ind)
        
        geese = observation['geese']
        self.current_goose_length = len(geese[observation['index']])  
        
        #### This is exception handling for if our goose died this turn, to use the last
        ### known index as its postion for the state centralizer
        if len(geese[observation['index']])>1:
            self.last_goose_ind = geese[observation['index']][0]
            
        ####
        biggest_goose = 0
        alive_geese = 0
        for ind, goose in enumerate(geese):
            if len(goose)>biggest_goose:
                biggest_goose = len(goose)
            if ind != observation['index'] and len(goose)>0:
                alive_geese+=1
        
        state = np.array([])
        state = np.append(state, self.__get_last_action_vec())
        state = np.append(state, self.current_goose_length/15)
        state = np.append(state, biggest_goose/15)
        state = np.append(state, alive_geese/8)
        state = np.append(state, self.step_count/200)
        state = np.append(state, board.flatten())
        
        state = state.flatten()
        
        return state
    
    def calculate_reward(self, observation):
        
        current_geese = observation['geese']
        prev = self.last_goose_length
        cur = len(current_geese[observation['index']])  
        
        reward = -1       
    
        ### If we grow, reward is 100
        if cur>prev:
            reward = 100
        
        else:
            reward = -1
        
        ### If we die -150
        if cur == 0:
            reward = -150
            
        ### see if any geese are alive

        alive_geese = 0
        for ind, goose in enumerate(current_geese):
            if ind != observation['index'] and len(goose)>0:
                alive_geese+=1
        
        # If we are the last one standing
        if alive_geese == 0 and cur>0:
            reward = 500
            
        ### if the game ends and we are the biggest
        if self.step_count == 200:
            biggest_goose = 0
            biggest_goose_ind = None
            for ind, goose in enumerate(current_geese):
                if len(goose)>biggest_goose:
                    biggest_goose = len(goose)
                    biggest_goose_ind = ind
            
            if biggest_goose_ind == observation['index']:
                reward = 500
        
        return reward
        

In [12]:
st_test = StateTranslator_Central()

In [13]:
env = make("hungry_geese", debug=True)
config = env.configuration

state_dict = env.reset(num_agents=4)[0]
observation = state_dict['observation']
action = state_dict['action']

In [14]:
state_dict

{'action': 'NORTH',
 'reward': 0,
 'info': {},
 'observation': {'remainingOverageTime': 60,
  'step': 0,
  'geese': [[60], [33], [4], [65]],
  'food': [70, 25],
  'index': 0},
 'status': 'ACTIVE'}

In [15]:
st_test.set_last_action("SOUTH")
st_test.update_step()
state_test = st_test.get_state(observation, config)
state_test

array([ 0.        ,  1.        ,  0.        ,  0.        ,  0.06666667,
        0.06666667,  0.375     ,  0.005     ,  0.        ,  0.        ,
        0.        ,  1.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        , -1.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        , -1.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  1.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        , -1.        ,  0.        ,  0.  

In [16]:
len(state_test)

85

In [17]:
st_test.set_last_action = 'NORTH'
st_test.translate_text_to_int('EAST')

1

In [18]:
st_test.calculate_reward(observation)

-1

In [19]:
class dqnAgent:
    """
    Given an environment state, choose an action, and learn from the reward
    https://towardsdatascience.com/reinforcement-learning-w-keras-openai-dqns-1eed3a5338c
    https://towardsdatascience.com/deep-q-learning-tutorial-mindqn-2a4c855abffc
    https://www.researchgate.net/post/What-are-possible-reasons-why-Q-loss-is-not-converging-in-Deep-Q-Learning-algorithm
    """

    def __init__(self, model=None, epsilon = 1.0, epsilon_min = 0.05):

        self.StateTrans = StateTranslator_Central()
        self.state_shape  = 85
        print('my state shape is:', self.state_shape)
        self.memory  = deque(maxlen=2000)
        self.gamma = 0.95
        self.epsilon = epsilon
        self.epsilon_min = epsilon_min
        self.epsilon_decay = 0.990
        self.learning_rate = 0.001
        self.tau = .125

        if model == None:
            self.model = self.create_model()
        else:
            self.model = model
        self.target_model = self.create_model()

    def create_model(self):
        model   = Sequential()
        model.add(Dense(2000, input_dim=self.state_shape, activation="relu"))
        model.add(Dense(1000, activation="relu"))
        model.add(Dense(500, activation="relu"))
        model.add(Dense(1000, activation="relu"))
        model.add(Dense(500, activation="relu"))
        model.add(Dense(100, activation="relu"))
        model.add(Dense(4))
        model.compile(loss="MSE",
            optimizer=Adam(lr=self.learning_rate))
        return model

    def act(self, state):
        self.epsilon *= self.epsilon_decay
        self.epsilon = max(self.epsilon_min, self.epsilon)
        if np.random.random() < self.epsilon:
            return random.choice([0,1,2,3])

        action_values = self.model.predict(state.reshape(-1, self.state_shape))[0]
        action = np.argmax(action_values)

        return action
    
    def translate_state(self, observation, configuration):
        state = self.StateTrans.get_state(observation, configuration)
        return state

    def __call__(self, observation, configuration):
        
        state = self.translate_state(observation, configuration)
        action = self.act(state)
        # State translator will take in 0, 1, 2, 3 and return straight, left or right, which in turn will 
        # be translated into a kaggle Action
        action_text = self.StateTrans.translate_action_to_text(action)
        
        # Update our step number and actions
        self.StateTrans.update_step()
        self.StateTrans.set_last_action(action_text)
        
        return action_text
    
    def remember(self, state, action, reward, new_state, done):
        self.memory.append([state, action, reward, new_state, done])

    def replay(self):
        batch_size = 32
        if len(self.memory) < batch_size:
            return

        samples = random.sample(self.memory, batch_size)
        ########################
        # This can be sped up significantly, but processing all samples in batch rather than 1 at a time
        ####################
        for sample in samples:
            state, action, reward, new_state, done = sample
            target = self.target_model.predict(state.reshape(-1, self.state_shape))
            if done:
                target[0][action] = reward
            else:
                Q_future = max(self.target_model.predict(new_state.reshape(-1, self.state_shape))[0])
                target[0][action] = reward + Q_future * self.gamma
            self.model.fit(state.reshape(-1, self.state_shape), target, epochs=1, verbose=0)

    def target_train(self):
        weights = self.model.get_weights()
        target_weights = self.target_model.get_weights()
        for i in range(len(target_weights)):
            target_weights[i] = weights[i] * self.tau + target_weights[i] * (1 - self.tau)
        self.target_model.set_weights(target_weights)

    def save_model(self, fn):
        self.model.save(fn)


### Training script

In [ ]:
steps_per_ep = 200
num_episodes = 10000

env = make("hungry_geese", debug=True)
config = env.configuration
train_name = 'central_agent'
model = keras.models.load_model('central_agent/trial-453')

dqn = dqnAgent(model = model, epsilon = 0.05)
agent2 = GreedyAgent()
agent3 = GreedyAgent()
agent4 = GreedyAgent()

agents = [dqn, agent2, agent3, agent4]

results_dic = {}
for ep in range(num_episodes):
    
    print('episode number: ', ep+453)
    state_dict = env.reset(num_agents=4)[0]
    observation = state_dict['observation']
    my_goose_ind = observation['index']
    
    action = state_dict['action']
    
    dqn.StateTrans.set_last_action(action)
    dqn.StateTrans.step_count = 0
    dqn.StateTrans.last_goose_length = 1
    cur_state = dqn.StateTrans.get_state(observation, config)
    
    
    done = False
    for step in range(steps_per_ep):
        actions = []
        for agent in agents:
            action = agent(observation, config)
            actions.append(action)
        
        state_dict = env.step(actions)[0]
        observation = state_dict['observation']
        print(observation)
        
        action = state_dict['action']
        status = state_dict['status']
        
        
        action_for_model = dqn.StateTrans.translate_text_to_int(action)
        new_state = dqn.StateTrans.get_state(observation, config)
        
        # Set rewards based on if value was gained or lost
        reward = dqn.StateTrans.calculate_reward(observation)
        # Update our goose length based on prev state
        dqn.StateTrans.update_length()
       

        if status != "ACTIVE":
            done = True
        
        
        #Temp for just training agent to go get food
        
#         if reward<-1:
#             reward = -1
        print('reward: ', reward)
        dqn.remember(cur_state, action_for_model, reward, new_state, done)
        
        cur_state = new_state
            
        # Check if my goose died

            
        if done:
            print('Done, Step: ', step)
            print('status, ', status)
            results_dic[ep] = reward
            
            if ep % 10 == 0:
                directory = train_name
                dqn.save_model(directory + f"/trial-{ep}")
                with open(directory + "/results_dic.pkl", 'wb') as f:
                    pickle.dump(results_dic, f)
            break
        

        if step%5 == 0:
            dqn.replay()        
            dqn.target_train()

#         if step%50 == 0:
#             print(f'We survived {step} steps')
#             directory = train_name
#             dqn.save_model(directory + f"/trial-{ep}")
        


my state shape is: 85
episode number:  453
{'remainingOverageTime': 60, 'step': 1, 'geese': [[54], [74], [73], [23]], 'food': [72, 5], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 2, 'geese': [[44], [8], [7], [34]], 'food': [72, 5], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 3, 'geese': [[55], [7], [6], [33]], 'food': [72, 5], 'index': 0}
reward:  -1
Goose Collision: WEST
{'remainingOverageTime': 60, 'step': 4, 'geese': [[66], [], [5, 6], [43]], 'food': [72, 45], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 5, 'geese': [[67], [], [71, 5], [32]], 'food': [72, 45], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 6, 'geese': [[68], [], [60, 71], [21]], 'food': [72, 45], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 7, 'geese': [[69], [], [49, 60], [10]], 'food': [72, 45], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 8, 'geese': [[70], [], [50, 49], [0]], 'food': [72, 45], 'index': 0}
reward:  -1


{'remainingOverageTime': 60, 'step': 17, 'geese': [[6, 5, 4, 70], [], [47], [45]], 'food': [10, 52], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 18, 'geese': [[7, 6, 5, 4], [], [48], [46]], 'food': [10, 52], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 19, 'geese': [[8, 7, 6, 5], [], [49], [47]], 'food': [10, 52], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 20, 'geese': [[9, 8, 7, 6], [], [50], [48]], 'food': [10, 52], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 21, 'geese': [[10, 9, 8, 7, 6], [], [51], [49]], 'food': [52, 34], 'index': 0}
reward:  100
{'remainingOverageTime': 60, 'step': 22, 'geese': [[21, 10, 9, 8, 7], [], [40], [38]], 'food': [52, 34], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 23, 'geese': [[11, 21, 10, 9, 8], [], [41], [39]], 'food': [52, 34], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 24, 'geese': [[12, 11, 21, 10, 9], [], [42], [40]], 'food': [52, 34], 'index':

{'remainingOverageTime': 60, 'step': 32, 'geese': [[53, 52], [51, 40], [44], [48, 37, 26]], 'food': [61, 71], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 33, 'geese': [[54, 53], [62, 51], [55], [59, 48, 37]], 'food': [61, 71], 'index': 0}
reward:  -1
Opposite action: (3, <Action.NORTH: 1>, <Action.SOUTH: 3>)
{'remainingOverageTime': 60, 'step': 34, 'geese': [[65, 54], [63, 62], [66], []], 'food': [61, 71], 'index': 0}
reward:  -1
Opposite action: (2, <Action.NORTH: 1>, <Action.SOUTH: 3>)
{'remainingOverageTime': 60, 'step': 35, 'geese': [[55, 65], [74, 63], [], []], 'food': [61, 71], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 36, 'geese': [[56, 55], [75, 74], [], []], 'food': [61, 71], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 37, 'geese': [[57, 56], [76, 75], [], []], 'food': [61, 71], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 38, 'geese': [[58, 57], [66, 76], [], []], 'food': [61, 71], 'index': 0}
reward:  -1
{'

{'remainingOverageTime': 60, 'step': 7, 'geese': [[66], [48], [50], [29]], 'food': [0, 34], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 8, 'geese': [[0, 66], [59], [61], [40]], 'food': [34, 7], 'index': 0}
reward:  100
{'remainingOverageTime': 60, 'step': 9, 'geese': [[11, 0], [60], [62], [41]], 'food': [34, 7], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 10, 'geese': [[22, 11], [61], [63], [42]], 'food': [34, 7], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 11, 'geese': [[23, 22], [62], [64], [43]], 'food': [34, 7], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 12, 'geese': [[34, 23, 22], [73], [75], [54]], 'food': [7, 36], 'index': 0}
reward:  100
{'remainingOverageTime': 60, 'step': 13, 'geese': [[35, 34, 23], [74], [76], [44]], 'food': [7, 36], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 14, 'geese': [[36, 35, 34, 23], [75], [66], [45]], 'food': [7, 58], 'index': 0}
reward:  100
{'remainingOverageTime

{'remainingOverageTime': 60, 'step': 7, 'geese': [[29, 28], [37], [41], [7]], 'food': [22, 71], 'index': 0}
reward:  -1
Opposite action: (3, <Action.NORTH: 1>, <Action.SOUTH: 3>)
{'remainingOverageTime': 60, 'step': 8, 'geese': [[40, 29], [48], [52], []], 'food': [22, 71], 'index': 0}
reward:  -1
Goose Collision: SOUTH
Goose Collision: WEST
{'remainingOverageTime': 60, 'step': 9, 'geese': [[], [47], [], []], 'food': [22, 71], 'index': 0}
reward:  -150
Done, Step:  8
status,  DONE
episode number:  463
{'remainingOverageTime': 60, 'step': 1, 'geese': [[9], [27], [51], [71]], 'food': [0, 6], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 2, 'geese': [[10], [26], [50], [70]], 'food': [0, 6], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 3, 'geese': [[0, 10], [25], [49], [69]], 'food': [6, 26], 'index': 0}
reward:  100
{'remainingOverageTime': 60, 'step': 4, 'geese': [[1, 0], [36], [60], [3]], 'food': [6, 26], 'index': 0}
reward:  -1
{'remainingOverageTime': 60,

...

{'remainingOverageTime': 60, 'step': 37, 'geese': [[38, 37, 36, 35, 34], [44, 55], [], [27]], 'food': [25, 10], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 38, 'geese': [[39, 38, 37, 36, 35], [45, 44], [], [28]], 'food': [25, 10], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 39, 'geese': [[40, 39, 38, 37, 36], [46, 45], [], [29]], 'food': [25, 10], 'index': 0}
reward:  -1
Goose Starved: Action.EAST
{'remainingOverageTime': 60, 'step': 40, 'geese': [[41, 40, 39, 38], [47], [], []], 'food': [25, 10], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 41, 'geese': [[42, 41, 40, 39], [36], [], []], 'food': [25, 10], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 42, 'geese': [[43, 42, 41, 40], [25, 36], [], []], 'food': [10, 20], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 43, 'geese': [[54, 43, 42, 41], [14, 25], [], []], 'food': [10, 20], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 44, 'geese': [[6

{'remainingOverageTime': 60, 'step': 12, 'geese': [[22, 32], [3], [29], [40]], 'food': [16, 15], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 13, 'geese': [[23, 22], [69], [18], [29]], 'food': [16, 15], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 14, 'geese': [[24, 23], [58], [7], [18]], 'food': [16, 15], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 15, 'geese': [[25, 24], [47], [73], [7]], 'food': [16, 15], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 16, 'geese': [[26, 25], [36], [62], [73]], 'food': [16, 15], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 17, 'geese': [[27, 26], [25], [51], [62]], 'food': [16, 15], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 18, 'geese': [[38, 27], [14], [40], [51]], 'food': [16, 15], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 19, 'geese': [[49, 38], [13], [39], [50]], 'food': [16, 15], 'index': 0}
reward:  -1
Goose Collision: WEST
{'rem

Goose Collision: SOUTH
Goose Collision: NORTH
{'remainingOverageTime': 60, 'step': 2, 'geese': [[], [13], [35], []], 'food': [14, 54], 'index': 0}
reward:  -150
Done, Step:  1
status,  DONE
episode number:  939
{'remainingOverageTime': 60, 'step': 1, 'geese': [[7, 73], [21], [8], [18]], 'food': [0, 67], 'index': 0}
reward:  100
Goose Collision: WEST
{'remainingOverageTime': 60, 'step': 2, 'geese': [[8, 7], [20], [], [17]], 'food': [0, 67], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 3, 'geese': [[19, 8], [9], [], [6]], 'food': [0, 67], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 4, 'geese': [[30, 19], [8], [], [5]], 'food': [0, 67], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 5, 'geese': [[41, 30], [7], [], [4]], 'food': [0, 67], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 6, 'geese': [[42, 41], [18], [], [15]], 'food': [0, 67], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 7, 'geese': [[43, 42], [29], [

{'remainingOverageTime': 60, 'step': 17, 'geese': [[36, 25], [24], [38, 27, 26], [68]], 'food': [60, 37], 'index': 0}
reward:  100
{'remainingOverageTime': 60, 'step': 18, 'geese': [[37, 36, 25], [35], [49, 38, 27], [2]], 'food': [60, 1], 'index': 0}
reward:  100
Opposite action: (1, <Action.NORTH: 1>, <Action.SOUTH: 3>)
{'remainingOverageTime': 60, 'step': 19, 'geese': [[48, 37, 36], [], [50, 49, 38], [1, 2]], 'food': [60, 21], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 20, 'geese': [[59, 48, 37], [], [61, 50, 49], [12, 1]], 'food': [60, 21], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 21, 'geese': [[70, 59, 48], [], [72, 61, 50], [23, 12]], 'food': [60, 21], 'index': 0}
reward:  -1
Goose Collision: EAST
Goose Collision: WEST
{'remainingOverageTime': 60, 'step': 22, 'geese': [[], [], [], [22, 23]], 'food': [60, 21], 'index': 0}
reward:  -150
Done, Step:  21
status,  DONE
episode number:  942
{'remainingOverageTime': 60, 'step': 1, 'geese': [[1, 67], 

{'remainingOverageTime': 60, 'step': 12, 'geese': [[34, 33], [54], [72], []], 'food': [13, 67], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 13, 'geese': [[45, 34], [43], [61], []], 'food': [13, 67], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 14, 'geese': [[56, 45], [33], [62], []], 'food': [13, 67], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 15, 'geese': [[67, 56, 45], [44], [73], []], 'food': [13, 30], 'index': 0}
reward:  100
{'remainingOverageTime': 60, 'step': 16, 'geese': [[68, 67, 56], [55], [7], []], 'food': [13, 30], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 17, 'geese': [[2, 68, 67], [66], [18], []], 'food': [13, 30], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 18, 'geese': [[13, 2, 68, 67], [0], [29], []], 'food': [30, 45], 'index': 0}
reward:  100
{'remainingOverageTime': 60, 'step': 19, 'geese': [[14, 13, 2, 68], [11], [40], []], 'food': [30, 45], 'index': 0}
reward:  -1
{'remainingOver

### Ideas for future improvement:
- Should get bonus reward with the action of actually collecting food
- Should be penalized harder for collision than starvation
- Add hunger feature

In [20]:
model = keras.models.load_model('central_agent/trial-453')


In [21]:
MyAgent = dqnAgent(model = model,
                   epsilon = 0,
                   epsilon_min = 0)
MyAgent.StateTrans.set_last_action('NORTH')

my state shape is: 85


In [22]:
env = make("hungry_geese", debug=True)


In [29]:
env.reset(num_agents=4)
results = env.run([MyAgent, GreedyAgent(), GreedyAgent(), GreedyAgent()])

Opposite action: (2, <Action.EAST: 2>, <Action.WEST: 4>)
Opposite action: (1, <Action.WEST: 4>, <Action.EAST: 2>)
Goose Collision: EAST


In [30]:
env.render(mode="ipython")

In [ ]:
results

In [ ]:
env.configuration